## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [1]:
import mlflow

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///workspaces/mlops-zoomcamp/02-experiment-tracking/scenarios/mlruns'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/02-experiment-tracking/scenarios/mlruns/0', creation_time=1748880289574, experiment_id='0', last_update_time=1748880289574, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/06/02 16:07:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp29ymtfbh/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/06/02 16:07:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'file:///workspaces/mlops-zoomcamp/02-experiment-tracking/scenarios/mlruns/579190979288619747/70d0d3f4679e4675be7a8534f9b6669c/artifacts'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/02-experiment-tracking/scenarios/mlruns/579190979288619747', creation_time=1748880378913, experiment_id='579190979288619747', last_update_time=1748880378913, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/02-experiment-tracking/scenarios/mlruns/0', creation_time=1748880289574, experiment_id='0', last_update_time=1748880289574, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [8]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")